## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thunder Bay,CA,48.4001,-89.3168,29.28,83,100,10.36,Clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,42.91,81,75,5.75,Clouds
2,2,Hithadhoo,MV,-0.6000,73.0833,82.99,75,29,10.40,Clouds
3,3,Rikitea,PF,-23.1203,-134.9692,78.48,63,2,10.27,Clear
4,4,Ust-Ishim,RU,57.6935,71.1665,32.72,74,88,4.18,Clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,78.48,63,2,10.27,Clear
7,7,Arraial Do Cabo,BR,-22.9661,-42.0278,72.18,71,0,7.38,Clear
9,9,Airai,TL,-8.9266,125.4092,73.99,60,100,1.41,Clouds
10,10,Bambous Virieux,MU,-20.3428,57.7575,76.21,94,75,1.14,Thunderstorm
21,21,Saint-Philippe,RE,-21.3585,55.7679,74.21,89,14,9.46,Rain
29,29,Atuona,PF,-9.8000,-139.0333,79.74,79,12,18.30,Clouds
46,46,Maragogi,BR,-9.0122,-35.2225,78.46,82,100,7.27,Clouds
49,49,Victoria,HK,22.2855,114.1577,74.17,82,96,1.01,Clouds
53,53,Carnarvon,AU,-24.8667,113.6333,73.47,68,88,16.11,Clouds
62,62,Mahebourg,MU,-20.4081,57.7000,76.24,94,75,1.14,Thunderstorm


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                155
City                   155
Country                155
Lat                    155
Lng                    155
Max Temp               155
Humidity               155
Cloudiness             155
Wind Speed             155
Current Description    155
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,78.48,63,2,10.27,Clear
7,7,Arraial Do Cabo,BR,-22.9661,-42.0278,72.18,71,0,7.38,Clear
9,9,Airai,TL,-8.9266,125.4092,73.99,60,100,1.41,Clouds
10,10,Bambous Virieux,MU,-20.3428,57.7575,76.21,94,75,1.14,Thunderstorm
21,21,Saint-Philippe,RE,-21.3585,55.7679,74.21,89,14,9.46,Rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,78.48,Clear,-23.1203,-134.9692,
7,Arraial Do Cabo,BR,72.18,Clear,-22.9661,-42.0278,
9,Airai,TL,73.99,Clouds,-8.9266,125.4092,
10,Bambous Virieux,MU,76.21,Thunderstorm,-20.3428,57.7575,
21,Saint-Philippe,RE,74.21,Rain,-21.3585,55.7679,
29,Atuona,PF,79.74,Clouds,-9.8000,-139.0333,
46,Maragogi,BR,78.46,Clouds,-9.0122,-35.2225,
49,Victoria,HK,74.17,Clouds,22.2855,114.1577,
53,Carnarvon,AU,73.47,Clouds,-24.8667,113.6333,
62,Mahebourg,MU,76.24,Thunderstorm,-20.4081,57.7000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

hotel_df.loc[(hotel_df["Hotel Name"] == ""), ["Hotel Name"]] = np.nan
clean_hotel_df=hotel_df.dropna(subset=['Hotel Name'])

clean_hotel_df.head()




,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,78.48,Clear,-23.1203,-134.9692,People ThankYou
7,Arraial Do Cabo,BR,72.18,Clear,-22.9661,-42.0278,Pousada Porto Praia
10,Bambous Virieux,MU,76.21,Thunderstorm,-20.3428,57.7575,Casa Tia Villa
21,Saint-Philippe,RE,74.21,Rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
29,Atuona,PF,79.74,Clouds,-9.8000,-139.0333,Villa Enata


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")





In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))